In [1]:
import requests
import pandas as pd
import numpy as np
import pprint
from datetime import date
import json
baseUrl="https://api.nhle.com/stats/rest/"
baseUrlEn="https://api.nhle.com/stats/rest/en/"

In [14]:
# res=requests.get(baseUrlEn + f"team")
# res.json()

# 'id': 12,
#    'franchiseId': 26,
#    'fullName': 'Carolina Hurricanes',
#    'leagueId': 133,
#    'rawTricode': 'CAR',
#    'triCode': 'CAR'},}

{'data': [{'id': 11,
   'franchiseId': 35,
   'fullName': 'Atlanta Thrashers',
   'leagueId': 133,
   'rawTricode': 'ATL',
   'triCode': 'ATL'},
  {'id': 34,
   'franchiseId': 26,
   'fullName': 'Hartford Whalers',
   'leagueId': 133,
   'rawTricode': 'HFD',
   'triCode': 'HFD'},
  {'id': 31,
   'franchiseId': 15,
   'fullName': 'Minnesota North Stars',
   'leagueId': 133,
   'rawTricode': 'MNS',
   'triCode': 'MNS'},
  {'id': 32,
   'franchiseId': 27,
   'fullName': 'Quebec Nordiques',
   'leagueId': 133,
   'rawTricode': 'QUE',
   'triCode': 'QUE'},
  {'id': 33,
   'franchiseId': 28,
   'fullName': 'Winnipeg Jets (1979)',
   'leagueId': 133,
   'rawTricode': 'WIN',
   'triCode': 'WIN'},
  {'id': 35,
   'franchiseId': 23,
   'fullName': 'Colorado Rockies',
   'leagueId': 133,
   'rawTricode': 'CLR',
   'triCode': 'CLR'},
  {'id': 36,
   'franchiseId': 3,
   'fullName': 'Ottawa Senators (1917)',
   'leagueId': 133,
   'rawTricode': 'SEN',
   'triCode': 'SEN'},
  {'id': 37,
   'franchis

In [3]:
homeGames=requests.get(baseUrlEn + f"game?&cayenneExp=season=20232024 and homeTeamId=12 and gameType=02")
# homeGames.json()
awayGames=requests.get(baseUrlEn + f"game?&cayenneExp=season=20232024 and visitingTeamId=12 and gameType=2")

data = pd.json_normalize(homeGames.json(),'data')
adata = pd.json_normalize(awayGames.json(),'data')
allGames=pd.concat([data,adata])
allGames.tail(10)

,id,easternStartTime,gameDate,gameNumber,gameScheduleStateId,gameStateId,gameType,homeScore,homeTeamId,period,season,visitingScore,visitingTeamId
31,2023021067,2024-03-16T19:00:00,2024-03-16,1067,1,7,2,4,10,5.0,20232024,5,12
32,2023021077,2024-03-17T18:00:00,2024-03-17,1077,1,7,2,2,9,3.0,20232024,7,12
33,2023021087,2024-03-19T19:30:00,2024-03-19,1087,1,7,2,1,2,3.0,20232024,4,12
34,2023021109,2024-03-22T19:00:00,2024-03-22,1109,1,7,2,7,15,5.0,20232024,6,12
35,2023021138,2024-03-26T19:00:00,2024-03-26,1138,1,7,2,4,5,3.0,20232024,1,12
36,2023021172,2024-03-30T19:00:00,2024-03-30,1172,1,1,2,0,8,NaN,20232024,0,12
37,2023021239,2024-04-09T19:00:00,2024-04-09,1239,1,1,2,0,6,NaN,20232024,0,12
38,2023021265,2024-04-12T20:00:00,2024-04-12,1265,1,1,2,0,19,NaN,20232024,0,12
39,2023021285,2024-04-14T18:00:00,2024-04-14,1285,1,1,2,0,16,NaN,20232024,0,12
40,2023021296,2024-04-16T19:00:00,2024-04-16,1296,1,1,2,0,29,NaN,20232024,0,12


In [2]:
#set date
datestart = '2023-10-10'
dateend = date.today()
#request to dict object  
res=requests.get(f"https://statsapi.web.nhl.com/api/v1/schedule?startDate={datestart}&endDate={dateend}").json()
# res=requests.get(f"https://statsapi.web.nhl.com/api/v1/schedule?startDate={datestart}&endDate={dateend}").json()
#print(res.items())
#normalize
data = pd.json_normalize(res,record_jpath= ['dates','games'])
data = data.drop_duplicates()

ConnectionError: HTTPSConnectionPool(host='statsapi.web.nhl.com', port=443): Max retries exceeded with url: /api/v1/schedule?startDate=2023-10-10&endDate=2024-02-19 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x167504b20>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [3]:
#get away team
df1 = data[['gameDate','teams.away.team.name']]
df1 = df1.rename(columns={"teams.away.team.name": "team"})
#get home team
df2 = data[['gameDate','teams.home.team.name']]
df2 = df2.rename(columns={"teams.home.team.name": "team"})
#concat  results
df3 = pd.concat([df1,df2],axis='rows', join='inner')
df3['datex'] = pd.to_datetime(df3['gameDate']).dt.date
df3['datex'] = df3.datex.astype('datetime64[ns]')
#clean and sort results
df3 = df3.drop('gameDate', axis=1)
df3 = df3.drop_duplicates()
df3 = df3.sort_values(by=['datex'])

In [4]:
data.head(5)

,gamePk,link,gameType,season,gameDate,status.abstractGameState,status.codedGameState,status.detailedState,status.statusCode,status.startTimeTBD,...,teams.home.leagueRecord.ot,teams.home.leagueRecord.type,teams.home.score,teams.home.team.id,teams.home.team.name,teams.home.team.link,venue.id,venue.name,venue.link,content.link
0,2023020001,/api/v1/game/2023020001/feed/live,R,20232024,2023-10-10T21:30:00Z,Final,7,Final,7,False,...,0,league,5,14,Tampa Bay Lightning,/api/v1/teams/14,5017.0,Amalie Arena,/api/v1/venues/5017,/api/v1/game/2023020001/content
1,2023020002,/api/v1/game/2023020002/feed/live,R,20232024,2023-10-11T00:00:00Z,Final,7,Final,7,False,...,0,league,2,5,Pittsburgh Penguins,/api/v1/teams/5,5034.0,PPG Paints Arena,/api/v1/venues/5034,/api/v1/game/2023020002/content
2,2023020003,/api/v1/game/2023020003/feed/live,R,20232024,2023-10-11T02:30:00Z,Final,7,Final,7,False,...,0,league,4,54,Vegas Golden Knights,/api/v1/teams/54,5178.0,T-Mobile Arena,/api/v1/venues/5178,/api/v1/game/2023020003/content
3,2023020004,/api/v1/game/2023020004/feed/live,R,20232024,2023-10-11T23:00:00Z,Final,7,Final,7,False,...,0,league,5,12,Carolina Hurricanes,/api/v1/teams/12,5066.0,PNC Arena,/api/v1/venues/5066,/api/v1/game/2023020004/content
4,2023020005,/api/v1/game/2023020005/feed/live,R,20232024,2023-10-11T23:00:00Z,Final,7,Final,7,False,...,0,league,6,10,Toronto Maple Leafs,/api/v1/teams/10,NaN,Scotiabank Arena,/api/v1/venues/null,/api/v1/game/2023020005/content


In [5]:

 options = [12] 
   
# selecting rows based on condition 
rslt_df = data[(data['teams.away.team.id'].isin(options)) |
          data['teams.home.team.id'].isin(options)] 
rslt_df[["gameDate","teams.away.score","teams.home.score","teams.away.team.name","teams.home.team.name","link"]]


,gameDate,teams.away.score,teams.home.score,teams.away.team.name,teams.home.team.name,link
3,2023-10-11T23:00:00Z,3,5,Ottawa Senators,Carolina Hurricanes,/api/v1/game/2023020004/feed/live
31,2023-10-15T02:30:00Z,6,5,Carolina Hurricanes,Los Angeles Kings,/api/v1/game/2023020032/feed/live
33,2023-10-16T00:30:00Z,3,6,Carolina Hurricanes,Anaheim Ducks,/api/v1/game/2023020034/feed/live
46,2023-10-18T02:30:00Z,6,3,Carolina Hurricanes,San Jose Sharks,/api/v1/game/2023020047/feed/live
59,2023-10-20T02:00:00Z,4,7,Carolina Hurricanes,Seattle Kraken,/api/v1/game/2023020060/feed/live
75,2023-10-22T01:00:00Z,4,6,Carolina Hurricanes,Colorado Avalanche,/api/v1/game/2023020076/feed/live
84,2023-10-24T23:00:00Z,0,3,Carolina Hurricanes,Tampa Bay Lightning,/api/v1/game/2023020086/feed/live
100,2023-10-26T23:00:00Z,2,3,Seattle Kraken,Carolina Hurricanes,/api/v1/game/2023020101/feed/live
111,2023-10-27T23:00:00Z,0,3,San Jose Sharks,Carolina Hurricanes,/api/v1/game/2023020112/feed/live
130,2023-10-30T23:00:00Z,3,2,Carolina Hurricanes,Philadelphia Flyers,/api/v1/game/2023020131/feed/live


In [6]:
	#/api/v1/teams/12
#teamRes=requests.get(f"https://statsapi.web.nhl.com/api/v1/teams/12").json()
#print(teamRes.items())
     

In [7]:
#	/api/v1/game/2023020001/feed/live
gameRes=requests.get(f"https://statsapi.web.nhl.com/api/v1/game/2023020001/feed/live").json()
json_formatted_str = json.dumps(gameRes, indent=2)

#print(json_formatted_str)
#print(gameRes.items())
#with open("sample.json", "w") as outfile:
#    outfile.write(json_formatted_str)

dataGame = pd.json_normalize(gameRes,record_path= ['liveData','plays','allPlays'])


In [8]:
dataGame.head(5)

,result.event,result.eventCode,result.eventTypeId,result.description,about.eventIdx,about.eventId,about.period,about.periodType,about.ordinalNum,about.periodTime,...,team.name,team.link,team.triCode,result.secondaryType,result.strength.code,result.strength.name,result.gameWinningGoal,result.emptyNet,result.penaltySeverity,result.penaltyMinutes
0,Game Scheduled,TBL1,GAME_SCHEDULED,Game Scheduled,0,1,1,REGULAR,1st,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Period Ready,TBL5,PERIOD_READY,Period Ready,1,5,1,REGULAR,1st,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Faceoff,TBL101,FACEOFF,Ryan O'Reilly faceoff won against Anthony Cirelli,3,101,1,REGULAR,1st,00:00,...,Nashville Predators,/api/v1/teams/18,NSH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Period Start,TBL102,PERIOD_START,Period Start,2,102,1,REGULAR,1st,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Stoppage,TBL8,STOP,Icing,4,8,1,REGULAR,1st,00:35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
